In [1]:
# load packages
import os
import sys
import re
import random
import numpy as np
import pandas as pd
import tensorflow as tf
tf.__version__

'1.12.0'

## Prepare Data

In [2]:
data = pd.read_csv("data/iris-dataset.csv") # load data
type(data)

pandas.core.frame.DataFrame

In [3]:
data.shape

(150, 6)

In [4]:
data.head(5)

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


In [5]:
target_values = data.Species.unique()
target_values

array(['Iris-setosa', 'Iris-versicolor', 'Iris-virginica'], dtype=object)

In [6]:
# create mapping string -> numeric value
target_values_mapping = dict()
i = 0
for tar in target_values:
    if tar not in target_values_mapping:
        target_values_mapping[tar] = i
        i += 1
    else:
        continue

In [7]:
target_values_mapping

{'Iris-setosa': 0, 'Iris-versicolor': 1, 'Iris-virginica': 2}

In [8]:
# convert string value to numeric values
data.Species = data.Species.apply(lambda x: target_values_mapping[x])
data.head(5)

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,0
1,2,4.9,3.0,1.4,0.2,0
2,3,4.7,3.2,1.3,0.2,0
3,4,4.6,3.1,1.5,0.2,0
4,5,5.0,3.6,1.4,0.2,0


In [9]:
data.Species.unique()

array([0, 1, 2])

In [10]:
features = np.array(data.iloc[:, 1:-1]) # load features into a numpy array
features.shape

(150, 4)

In [11]:
labels = np.array(data.iloc[:, [-1]]) # load target values into a numpy array
labels.shape

(150, 1)

In [12]:
features[0], labels[0]

(array([5.1, 3.5, 1.4, 0.2]), array([0]))

In [13]:
# split data into train and validate
val_split_size = 0.2
random_indices = np.random.rand(len(features)) < val_split_size

# split data
train_x = features[~random_indices]
val_x = features[random_indices]
train_y = labels[~random_indices]
val_y = labels[random_indices]

## Vectorized Logistic Regression Model

In [14]:
# placeholders
x = tf.placeholder(dtype=tf.float32, shape=(None, features.shape[1]), name="feature_x")
y = tf.placeholder(dtype=tf.float32, shape=(None, 1), name="target_y")

In [15]:
# define weights and bias
w = tf.Variable(tf.zeros(shape=(1, features.shape[1])))
b = tf.Variable(tf.zeros(shape=(1, features.shape[1])))

In [16]:
logistic_model = tf.sigmoid(tf.add(tf.matmul(x, w, transpose_b=True), b))

In [17]:
loss_function = tf.reduce_mean(-tf.reduce_sum(y*tf.log(logistic_model), keepdims=True))

In [18]:
learning_rate = 0.001

In [19]:
# compute gradient
grad_w, grad_b = tf.gradients(xs=[w, b], ys=loss_function)

new_w = w.assign(w - learning_rate*grad_w) # update weights
new_b = b.assign(b - learning_rate*grad_b) # update bias

In [20]:
# train the model
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print("[Training...]")
    for epoch in range(10):
        _, _, c_loss = sess.run(
            [new_w, new_b, loss_function], feed_dict={x:train_x, y:train_y}
        )
        print("Step:", epoch, "\tTrain Loss:", np.round(c_loss, decimals=4))
    val_loss = sess.run(loss_function, feed_dict={x:val_x, y:val_y})
    print("Validation Loss:", np.round(val_loss, decimals=4))

[Training...]
Step: 0 	Train Loss: 332.7108
Step: 1 	Train Loss: 0.0
Step: 2 	Train Loss: 0.0
Step: 3 	Train Loss: 0.0
Step: 4 	Train Loss: 0.0
Step: 5 	Train Loss: 0.0
Step: 6 	Train Loss: 0.0
Step: 7 	Train Loss: 0.0
Step: 8 	Train Loss: 0.0
Step: 9 	Train Loss: 0.0
Validation Loss: 0.0


Perfect Score!!